In [1]:
%matplotlib inline

In [5]:
import pandas as pd
import akshare as ak
import quantstats as qs
from IPython.display import display

In [3]:
fund_name_em_df = ak.fund_name_em()
display(fund_name_em_df.sample(10))

,基金代码,拼音缩写,基金简称,基金类型,拼音全称
726,000995,JXRYLHPZHHC,建信睿盈灵活配置混合C,混合型-灵活,JIANXINRUIYINGLINGHUOPEIZHIHUNHEC
7567,009576,DFHZYSNCYHH,东方红智远三年持有混合,混合型-偏股,DONGFANGHONGZHIYUANSANNIANCHIYOUHUNHE
385,000559,NATTBA,诺安天天宝A,货币型,NUOANTIANTIANBAOA
7925,009953,HTBRJQZQ,华泰柏瑞锦乾债券,债券型-长债,HUATAIBAIRUIJINQIANZHAIQUAN
10554,012719,HXXXJJYNCYHHA,华夏新兴经济一年持有混合A,混合型-偏股,HUAXIAXINXINGJINGJIYINIANCHIYOUHUNHEA
12674,015043,XBLDSDDLHHFQA,西部利得时代动力混合发起A,混合型-偏股,XIBULIDESHIDAIDONGLIHUNHEFAQIA
12173,014437,PHZZTYCDAAAZS7TCY,鹏华中证同业存单AAA指数7天持有,混合型-偏债,PENGHUAZHONGZHENGTONGYECUNDANAAAZHISHU7TIANCHIYOU
11498,013711,GFZRZZTLOFC,广发再融资主题(LOF)C,混合型-灵活,GUANGFAZAIRONGZIZHUTILOFC
14230,202025,NFSZ380ETFLJA,南方上证380ETF联接A,指数型-股票,NANFANGSHANGZHENG380ETFLIANJIEA
7568,009577,SYJYYYNDKZQ,上银聚永益一年定开债券,债券型-长债,SHANGYINJUYONGYIYINIANDINGKAIZHAIQUAN


In [9]:
fund_type = fund_name_em_df['基金类型'].drop_duplicates()
fund_type

0            混合型-偏股
2           债券型-可转债
4            债券型-长债
6            指数型-股票
7               货币型
12          债券型-混合债
21           混合型-灵活
28             QDII
65              股票型
67          债券型-中短债
81           混合型-偏债
168      商品（不含QDII）
291         混合-绝对收益
327             理财型
1085         混合型-平衡
3670            FOF
14185         Reits
Name: 基金类型, dtype: object

In [10]:
fund_name_em_df.groupby(fund_name_em_df['基金类型']).count()

,基金代码,拼音缩写,基金简称,拼音全称
基金类型,,,,
FOF,422,422,422,422
QDII,407,407,407,407
Reits,12,12,12,12
债券型-中短债,603,603,603,603
债券型-可转债,74,74,74,74
债券型-混合债,1044,1044,1044,1044
债券型-长债,2950,2950,2950,2950
商品（不含QDII）,47,47,47,47
指数型-股票,2051,2051,2051,2051


In [4]:
equity_fund_df = fund_name_em_df[fund_name_em_df['基金类型'] == '股票型'].loc[:, ['基金代码', '基金简称']]
# df = equity_fund_df[equity_fund_df['基金简称'].str.contains('ESG')]
# display(df)
equity_fund_df.drop([318,348,871,12355,12356,15454], inplace=True)
equity_fund_df

,基金代码,基金简称
65,000082,嘉实研究阿尔法股票
221,000309,大摩品质生活精选股票
232,000326,南方中小盘成长股票
288,000409,鹏华环保产业股票
290,000411,景顺长城优质成长股票
...,...,...
16018,920003,中金新锐股票A
16024,920922,中金精选股票C
16025,920923,中金新锐股票C
16089,970041,国海量化优选一年持有股票A


In [2]:
def GetFundInfoByAkshare(fund, indicator):
    fund_open_fund_info_em_df = ak.fund_open_fund_info_em(fund=fund, indicator=indicator)
    return fund_open_fund_info_em_df

In [3]:
def FormatData(fund_open_fund_info_em_df):
    data = fund_open_fund_info_em_df
    data.rename(columns={'净值日期':'date', '累计净值':'cumvalue'}, inplace=True)
    data.date = pd.to_datetime(data['date'])
    data.set_index('date', inplace=True)
    cumvalue = pd.Series(data.cumvalue)
    return cumvalue

In [7]:
fund_matrix = equity_fund_df.reset_index()
fund_matrix.drop('index',axis=1, inplace=True)
fund_matrix.rename(columns={'基金代码':'fund_code', '基金简称':'fund_abb'}, inplace=True)
display(fund_matrix)

,fund_code,fund_abb
0,000082,嘉实研究阿尔法股票
1,000309,大摩品质生活精选股票
2,000326,南方中小盘成长股票
3,000409,鹏华环保产业股票
4,000411,景顺长城优质成长股票
...,...,...
795,920003,中金新锐股票A
796,920922,中金精选股票C
797,920923,中金新锐股票C
798,970041,国海量化优选一年持有股票A


In [68]:
for index in fund_matrix.index:
    fund = fund_matrix.loc[index, 'fund_code']
    fund_open_fund_info_em_df = GetFundInfoByAkshare(fund=fund, indicator="累计净值走势")
    cumvalue = FormatData(fund_open_fund_info_em_df)
    returns = cumvalue.pct_change().dropna()
    sharpe_ratio = qs.stats.sharpe(returns=returns)
    sortino_ratio = qs.stats.sortino(returns=returns)
    max_drawdown = qs.stats.max_drawdown(cumvalue)
    var = qs.stats.var(returns=returns)
    cvar = qs.stats.cvar(returns=returns)
    volatility = qs.stats.volatility(returns=returns)
    fund_matrix.loc[index, 'sharpe'] = sharpe_ratio
    fund_matrix.loc[index, 'sortino'] = sortino_ratio
    fund_matrix.loc[index, 'max_drawdown'] = max_drawdown
    fund_matrix.loc[index, 'var'] = var
    fund_matrix.loc[index, 'cvar'] = cvar
    fund_matrix.loc[index, 'volatility'] = volatility

display(fund_matrix)


C:\Users\ThinkPad\.conda\envs\qtconda\lib\site-packages\quantstats\stats.py:341: RuntimeWarning: invalid value encountered in double_scalars
  downside = _np.sqrt((returns[returns < 0] ** 2).sum() / len(returns))
C:\Users\ThinkPad\.conda\envs\qtconda\lib\site-packages\quantstats\stats.py:629: RuntimeWarning: Mean of empty slice.
  c_var = returns[returns < var].values.mean()
C:\Users\ThinkPad\.conda\envs\qtconda\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\ThinkPad\.conda\envs\qtconda\lib\site-packages\quantstats\stats.py:341: RuntimeWarning: invalid value encountered in double_scalars
  downside = _np.sqrt((returns[returns < 0] ** 2).sum() / len(returns))
C:\Users\ThinkPad\.conda\envs\qtconda\lib\site-packages\quantstats\stats.py:629: RuntimeWarning: Mean of empty slice.
  c_var = returns[returns < var].values.mean()
C:\Users\ThinkPad\.conda\envs\qtconda\lib\site-packages\numpy\c

,fund_code,fund_abb,sharpe,sortino,max_drawdown,var,cvar,volatility
0,000082,嘉实研究阿尔法股票,0.671314,0.912618,-0.431181,-0.019135,-0.035152,0.189548
1,000309,大摩品质生活精选股票,0.680630,0.924815,-0.569028,-0.028378,-0.048255,0.281205
2,000326,南方中小盘成长股票,0.916061,1.302787,-0.227615,-0.013083,-0.021236,0.130858
3,000409,鹏华环保产业股票,0.868338,1.222053,-0.491221,-0.028100,-0.045264,0.280522
4,000411,景顺长城优质成长股票,0.460136,0.630492,-0.497608,-0.023474,-0.041226,0.230615
...,...,...,...,...,...,...,...,...
795,920003,中金新锐股票A,1.200071,1.740833,-0.296932,-0.019216,-0.025922,0.194384
796,920922,中金精选股票C,-0.243946,-0.321725,-0.337647,-0.019303,-0.026547,0.184570
797,920923,中金新锐股票C,1.208971,1.761144,-0.330795,-0.022207,-0.030583,0.224730
798,970041,国海量化优选一年持有股票A,-0.319185,-0.396108,-0.247322,-0.021488,-0.030855,0.204876


In [82]:
fund_matrix.dropna(inplace=True)
fund_matrix.to_excel('data/fund_matrix.xlsx')
display(fund_matrix)

,fund_code,fund_abb,sharpe,sortino,max_drawdown,var,cvar,volatility
0,000082,嘉实研究阿尔法股票,0.671314,0.912618,-0.431181,-0.019135,-0.035152,0.189548
1,000309,大摩品质生活精选股票,0.680630,0.924815,-0.569028,-0.028378,-0.048255,0.281205
2,000326,南方中小盘成长股票,0.916061,1.302787,-0.227615,-0.013083,-0.021236,0.130858
3,000409,鹏华环保产业股票,0.868338,1.222053,-0.491221,-0.028100,-0.045264,0.280522
4,000411,景顺长城优质成长股票,0.460136,0.630492,-0.497608,-0.023474,-0.041226,0.230615
...,...,...,...,...,...,...,...,...
795,920003,中金新锐股票A,1.200071,1.740833,-0.296932,-0.019216,-0.025922,0.194384
796,920922,中金精选股票C,-0.243946,-0.321725,-0.337647,-0.019303,-0.026547,0.184570
797,920923,中金新锐股票C,1.208971,1.761144,-0.330795,-0.022207,-0.030583,0.224730
798,970041,国海量化优选一年持有股票A,-0.319185,-0.396108,-0.247322,-0.021488,-0.030855,0.204876


In [6]:
fund_matrix = pd.read_excel('data/fund_matrix.xlsx',
                            usecols=['fund_code','fund_abb'],
                            dtype={'fund_code':str})
fund_matrix

,fund_code,fund_abb
0,000082,嘉实研究阿尔法股票
1,000309,大摩品质生活精选股票
2,000326,南方中小盘成长股票
3,000409,鹏华环保产业股票
4,000411,景顺长城优质成长股票
...,...,...
789,920003,中金新锐股票A
790,920922,中金精选股票C
791,920923,中金新锐股票C
792,970041,国海量化优选一年持有股票A


In [ ]:
for index in fund_matrix.index:
    fund = fund_matrix.loc[index, 'fund_code']
    fund_open_fund_info_em_df = GetFundInfoByAkshare(fund=fund, indicator="累计净值走势")
    cumvalue = FormatData(fund_open_fund_info_em_df)
    returns = cumvalue.pct_change().dropna()
    qs.reports.html(returns=cumvalue,
                    output='E:\\Project\\portfolioanalysis\\stock-debt-hybrid-strategy\\data\\',
                    title=fund_matrix.loc[index, 'fund_abb'] + '概览',
                    download_filename=fund_matrix.loc[index, 'fund_abb'] + '概览.html')

In [79]:
help(qs.reports.html)

Help on function html in module quantstats.reports:

html(returns, benchmark=None, rf=0.0, grayscale=False, title='Strategy Tearsheet', output=None, compounded=True, periods_per_year=252, download_filename='quantstats-tearsheet.html', figfmt='svg', template_path=None, match_dates=False, **kwargs)

